In [1]:
#!pip install soundfile

In [2]:
import pyaudio
import sounddevice as sd
import soundfile as sf

# See the microphone

In [3]:
sd.query_devices() 

< 0 Volumio, Core Audio (0 in, 2 out)
  1 Built-in Microph, Core Audio (2 in, 0 out)
  2 Built-in Output, Core Audio (0 in, 2 out)
  3 Samson Meteor Mic, Core Audio (0 in, 2 out)
> 4 Samson Meteor Mic, Core Audio (2 in, 0 out)
  5 Soundflower (2ch), Core Audio (2 in, 2 out)
  6 Soundflower (64ch), Core Audio (64 in, 64 out)
  7 WavTap, Core Audio (2 in, 2 out)
  8 System Audio Recorder, Core Audio (2 in, 0 out)

In [4]:
sd.query_devices?

In [5]:
#sd.default.device =7  # record device take one with (2 in, ..)
sd.default.channels = 2

##### Record for 10 seconds

In [14]:
duration = 10  # seconds
fs=44100
myrecording = sd.rec(int(duration * fs) )

##### Play the recording

In [17]:
sd.play(myrecording)

##### Format
The format is a numpy array

In [15]:
print myrecording

[[ 0.01452637  0.01446533]
 [ 0.01425171  0.01419067]
 [ 0.01248169  0.01235962]
 ..., 
 [-0.02587891 -0.02584839]
 [-0.02297974 -0.02297974]
 [-0.02102661 -0.02099609]]


In [18]:
myrecording[:, 0]# left channel
myrecording[:, 1]# right channel

array([ 0.01446533,  0.01419067,  0.01235962, ..., -0.02584839,
       -0.02297974, -0.02099609], dtype=float32)

##### Save the recording 

In [10]:
sf.write?

In [19]:
sf.write("myWave.wav",myrecording,fs)

##### Read the recording 

In [20]:

data, fs = sf.read('myWave.wav', dtype='float32')
sd.play(data, fs, device=0, blocking=True)
 

#####  Another way

In [ ]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output3.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [ ]:
sf.write?

In [ ]:
#def callback(indata: numpy.ndarray, frames: int, time: CData, status: CallbackFlags) -> None

def callback(outdata, frame_count, time, status):
    view = sf.read(frame_count, dtype='float32', out=outdata)
    if (view.size != outdata.size):   # this was the last buffer.
        # zero the part of outdata not written into  by sf.read()
        outdata[view.shape[0]:,:].fill(0.0)
        print('stopping')
        raise sd.CallbackStop()
#with sf.SoundFile("mywav.wav", mode='x', samplerate=44100, channels=2, subtype=2) as mywav:
with sf.SoundFile("mywav6.wav", mode='x', samplerate=44100,channels=2) as mywav:
#    with sd.InputStream(samplerate=44100, device=0, channels=2, callback=callback):
    with sd.InputStream(samplerate=44100, device=0, channels=2, callback=callback):
        print('#' * 80)
        print('press Ctrl+C to stop the recording')
        print('#' * 80)
        while True:
            mywav.write('test.wav')